In [1]:
import pandas as pd

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)

In [2]:
df = pd.read_parquet("muestreos_parcelas.parquet")

estados = [
    "estado_fenologico_1",
    "estado_fenologico_2",
    "estado_fenologico_3",
    "estado_fenologico_4",
    "estado_fenologico_5",
    "estado_fenologico_6",
    "estado_fenologico_7",
    "estado_fenologico_8",
    "estado_fenologico_9",
    "estado_fenologico_10",
    "estado_fenologico_11",
    "estado_fenologico_12",
    "estado_fenologico_13",
    "estado_fenologico_14",
]

In [3]:
def find_estado_with_value_two(row) -> int:
    """
    Returns numero del siguiente etado (yt+1). (e.g., 13)
    """
    for column in row.index:
        if row[column] == 2:
            number_growth_stage = int(column.split("_")[-1])
            return number_growth_stage


def get_valid_dataset(df: pd.DataFrame(), max_days_till_next_date: int) -> pd.DataFrame():
    
    # Removing null and filling empty estados with 0
    df = df.dropna(subset=estados, how="all")
    df = df.dropna(subset=["codparcela"])
    df[estados] = df[estados].fillna(0)

    #Filtering Dataset to keep rows only with one unique 2
    df["count_2s"] = df[estados].eq(2).sum(axis=1)
    df = df[df.count_2s == 1]

    # Sorting by date
    df["fecha"] = pd.to_datetime(df["fecha"])
    df.sort_values(by="fecha", inplace=True)

    # Creating a column to display the number of days till the next observation
    df["next_date"] = df.groupby("codparcela", observed=True)["fecha"].shift(-1)
    df["days_until_next_visit"] = (df["next_date"] - df["fecha"]).dt.days

    # Creating a column to display the next estado_fenológico (yt+1)
    df["estado_actual"] = df[estados].apply(find_estado_with_value_two, axis=1)
    df["next_estado"] = df.groupby("codparcela", observed=True)["estado_actual"].shift(-1)

    # Removing the parcels with only one entry and the last entry for every parcel
    df = df.dropna(subset=["days_until_next_visit"])  # 5150 entries removed

    #Changing datatypes
    df["days_until_next_visit"] = df["days_until_next_visit"].astype("int16")
    df["next_estado"] = df["next_estado"].astype("int8")
    df[estados] = df[estados].astype("int8")

    # Filtering the max days
    df = df[df["days_until_next_visit"] < max_days_till_next_date]

    excluded_columns = ['estado_actual','count_2s', 'next_date']

    return df.loc[:, [i for i in df.columns if i not in excluded_columns]]


In [4]:
df_new = get_valid_dataset(df, 10)

In [5]:
df_new[df_new["102_coordenada_x_(utm)"].isnull()].codparcela.value_counts()

codparcela
001-00048-00-50    172
019-00160-00-10    112
075-00019-00-00    107
014-00018-00-00    106
005-00044-00-00    101
                  ... 
009-00107-01-01      0
009-00106-03-01      0
009-00106-02-00      0
009-00106-00-00      0
015-00013-01         0
Name: count, Length: 5239, dtype: int64

In [6]:
df_new[df_new['codparcela']=='001-00048-00-50']["102_coordenada_x_(utm)"].value_counts()

102_coordenada_x_(utm)
435359.0    40
435109.0    20
Name: count, dtype: int64